In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score
import numpy as np
import itertools

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def load_test_data(loc, date, b1, b2):
    x = np.load(f"/data/GPM_HIM8/exp_paper/X_B{b1}_{loc}_{date}.npy")[:,::8,::8].flatten()[:,None]
    x = np.concatenate((x, np.load(f"/data/GPM_HIM8/exp_paper/X_B{b2}_{loc}_{date}.npy")[:,::8,::8].flatten()[:,None]), axis=1)
    y = np.clip(np.load(f"/data/GPM_HIM8/exp_paper/Y_{loc}_{date}.npy")[:,::2,::2].flatten(),0,40)
    
    return x, y


def load_train_data(loc, dates, b1, b2):
    x = np.load(f"/data/GPM_HIM8/exp_paper/X_B{b1}_{loc}_{dates[0]}.npy")[:,::8,::8].flatten()[:,None]
    x = np.concatenate((x, np.load(f"/data/GPM_HIM8/exp_paper/X_B{b2}_{loc}_{dates[0]}.npy")[:,::8,::8].flatten()[:,None]), axis=1)
    y = np.clip(np.load(f"/data/GPM_HIM8/exp_paper/Y_{loc}_{dates[0]}.npy")[:,::2,::2].flatten(),0,40)
    
    for d in dates[1:]:
        xi = np.load(f"/data/GPM_HIM8/exp_paper/X_B{b1}_{loc}_{d}.npy")[:,::8,::8].flatten()[:,None]
        xi = np.concatenate((xi, np.load(f"/data/GPM_HIM8/exp_paper/X_B{b2}_{loc}_{d}.npy")[:,::8,::8].flatten()[:,None]), axis=1)
        x = np.concatenate((x,xi), axis=0)
        yi = np.clip(np.load(f"/data/GPM_HIM8/exp_paper/Y_{loc}_{d}.npy")[:,::2,::2].flatten(),0,40)
        y = np.concatenate((y,yi), axis=0)
        
    return x, y

In [4]:
#Save models for comparison

import pickle

dates = ["201811","201812","201901","201902"]

for loc in ["SYD","NT","WA"]:
    for i, d in enumerate(dates):
        print(loc, d)
        x_train, y_train = load_train_data(loc, dates[:i]+dates[i+1:], 11, 16)
        rf = RandomForestRegressor(max_depth=6, random_state=0, n_jobs=8)
        rf.fit(x_train, y_train)

        pickle.dump(rf, open(f'rf_{loc}_{d}.pkl', 'bw'))

SYD 201811
SYD 201812
SYD 201901
SYD 201902
NT 201811
NT 201812
NT 201901
NT 201902
WA 201811
WA 201812
WA 201901
WA 201902


In [29]:
dates = ["201811","201812","201901","201902"]

print("model,mse,f1_01,f1_1")

for loc in ["SYD","NT","WA"]:
    for i, d in enumerate(dates):
        for bands in itertools.combinations([8,9,10,11,12,13,14,15,16], 2):
            x_train, y_train = load_train_data(loc, dates[:i]+dates[i+1:], bands[0], bands[1])
            rf = RandomForestRegressor(max_depth=6, random_state=0, n_jobs=8)
            rf.fit(x_train, y_train)
            
            x_test, y_test = load_test_data(loc, d, bands[0], bands[1])
            y_hat = rf.predict(x_test)
            
            print(f"{loc}_{d}_b{bands[0]}_b{bands[1]},{np.mean(np.square(y_hat-y_test))},{f1_score(y_hat>.2,y_test>.2)},{f1_score(y_hat>1.,y_test>1.)}")

model,mse,f1_01,f1_1
SYD_201811_b8_b9,0.6633938638251494,0.4368719843570299,0.3653409142763272
SYD_201811_b8_b10,0.6528174001997902,0.48172706513970964,0.38404044149232003
SYD_201811_b8_b11,0.6518014678615018,0.5046877383426944,0.4109374131973378
SYD_201811_b8_b12,0.826778231610231,0.4478290141121965,0.41425502687453325
SYD_201811_b8_b13,0.6638247102971643,0.5168007442348633,0.42046970530055927
SYD_201811_b8_b14,0.657223494049485,0.5056198761775961,0.4128133585164835
SYD_201811_b8_b15,0.6653306139157753,0.5043661479146847,0.4084002223362772
SYD_201811_b8_b16,0.6650879531821187,0.49076608450659626,0.3958091500082303
SYD_201811_b9_b10,0.6652199731570076,0.47588191282798165,0.3632991579524217
SYD_201811_b9_b11,0.6499687799762939,0.5002032158284451,0.4145127669030792
SYD_201811_b9_b12,0.8039161958003556,0.4409668097664713,0.4004643064422519
SYD_201811_b9_b13,0.6609545622255489,0.5148675717130482,0.4151203364814483
SYD_201811_b9_b14,0.6609238052486465,0.5088591110044896,0.39420698327353143


SYD_201902_b8_b9,0.3833337467026258,0.3521702596872062,0.3364917329568593
SYD_201902_b8_b10,0.37372258251941515,0.36197517148312364,0.36498820770429985
SYD_201902_b8_b11,0.36606409307395504,0.4178581790602619,0.39346615050514583
SYD_201902_b8_b12,0.3851366083457628,0.4288836870417164,0.33776817462612924
SYD_201902_b8_b13,0.364329994448452,0.4158885770422595,0.40039856968978416
SYD_201902_b8_b14,0.3719856197774258,0.4102723054937454,0.39098666143585714
SYD_201902_b8_b15,0.3742784598324925,0.3999467873664339,0.38840547305396655
SYD_201902_b8_b16,0.37962704210717163,0.389173360917658,0.3774351363927911
SYD_201902_b9_b10,0.387172515680282,0.37658505815099547,0.29802112658716157
SYD_201902_b9_b11,0.3742139880096599,0.4338023503658045,0.38518684534913616
SYD_201902_b9_b12,0.38959037888690695,0.4424303471309029,0.23775025945368625
SYD_201902_b9_b13,0.37347971614026515,0.43739043919078985,0.37847599410490773
SYD_201902_b9_b14,0.3805352060720217,0.42723739861929755,0.37575558259611275
SYD_20190

NT_201901_b8_b10,1.727159725331522,0.5526327866732924,0.5588206009328781
NT_201901_b8_b11,1.7056875406167737,0.5861651433878196,0.5728665023384533
NT_201901_b8_b12,1.6454990134220562,0.5862504175155085,0.574554850905543
NT_201901_b8_b13,1.7103900701009642,0.5911425633407082,0.5709766802570557
NT_201901_b8_b14,1.7242180460417702,0.5821752105275148,0.5663141710817589
NT_201901_b8_b15,1.7305882820415897,0.5799121596416383,0.5563122661163716
NT_201901_b8_b16,1.7375171073625693,0.5759318864238829,0.5503644306000025
NT_201901_b9_b10,1.7342029545658306,0.5503484464434323,0.5563233556117043
NT_201901_b9_b11,1.7066182000202479,0.5863011811631745,0.5708007705209807
NT_201901_b9_b12,1.6487014384855294,0.5874953626532881,0.5719655559138306
NT_201901_b9_b13,1.7112274693623193,0.5916920577619196,0.5692678062135158
NT_201901_b9_b14,1.7268967471358148,0.58878941587423,0.561699648663519
NT_201901_b9_b15,1.730552679665194,0.5778473670595463,0.5512341097890882
NT_201901_b9_b16,1.7390846685217816,0.577687

WA_201812_b8_b12,0.13463067550680413,0.4774286380171654,0.3486277300831814
WA_201812_b8_b13,0.13443373599408076,0.4826168214444816,0.35379053364680835
WA_201812_b8_b14,0.1353674070351129,0.47208795348592414,0.3600565955594253
WA_201812_b8_b15,0.13604082456486802,0.4643423389142567,0.34518062167460095
WA_201812_b8_b16,0.1375321897411574,0.4505517122150539,0.33145464917284656
WA_201812_b9_b10,0.1383927744464477,0.439846486709252,0.31194915551810104
WA_201812_b9_b11,0.1347735598568935,0.4864579565700813,0.35238084786497376
WA_201812_b9_b12,0.13512917457600976,0.4856719354763345,0.32329021404953884
WA_201812_b9_b13,0.13460442159779834,0.4925031322249306,0.34340542595781803
WA_201812_b9_b14,0.1352968023808235,0.48232409956199906,0.3462226442594351
WA_201812_b9_b15,0.1359507902750774,0.47201570562083284,0.32878476094129006
WA_201812_b9_b16,0.13767318662815736,0.4599785750043447,0.30237414153118247
WA_201812_b10_b11,0.13507389946105508,0.49337584594914335,0.35418005205327296
WA_201812_b10_b12

In [ ]:
model,mse,f1_01,f1_1
SYD_201811_b8_b9,0.6633938638251494,0.4368719843570299,0.3653409142763272
SYD_201811_b8_b10,0.6528174001997902,0.48172706513970964,0.38404044149232003
SYD_201811_b8_b11,0.6518014678615018,0.5046877383426944,0.4109374131973378
SYD_201811_b8_b12,0.826778231610231,0.4478290141121965,0.41425502687453325
SYD_201811_b8_b13,0.6638247102971642,0.5168007442348633,0.42046970530055927
SYD_201811_b8_b14,0.657223494049485,0.5056198761775961,0.4128133585164835
SYD_201811_b8_b15,0.6653306139157753,0.5043661479146847,0.4084002223362772
SYD_201811_b8_b16,0.6650879531821187,0.49076608450659626,0.3958091500082303
SYD_201811_b9_b10,0.6652199731570076,0.47588191282798165,0.3632991579524217
SYD_201811_b9_b11,0.6499687799762939,0.5002032158284451,0.4145127669030792

In [ ]:
model,mse,f1_01,f1_1
SYD_201811_b8_b9,0.6633938638251494,0.4368719843570299,0.3653409142763272
SYD_201811_b8_b10,0.6528174001997902,0.48172706513970964,0.38404044149232003
SYD_201811_b8_b11,0.6518014678615018,0.5046877383426944,0.4109374131973378
SYD_201811_b8_b12,0.826778231610231,0.4478290141121965,0.41425502687453325
SYD_201811_b8_b13,0.6638247102971643,0.5168007442348633,0.42046970530055927
SYD_201811_b8_b14,0.657223494049485,0.5056198761775961,0.4128133585164835
SYD_201811_b8_b15,0.6653306139157753,0.5043661479146847,0.4084002223362772
SYD_201811_b8_b16,0.6650879531821187,0.49076608450659626,0.3958091500082303
SYD_201811_b9_b10,0.6652199731570076,0.47588191282798165,0.3632991579524217
SYD_201811_b9_b11,0.6499687799762939,0.5002032158284451,0.4145127669030792
SYD_201811_b9_b12,0.8039161958003556,0.4409668097664713,0.4004643064422519
SYD_201811_b9_b13,0.6609545622255489,0.5148675717130482,0.4151203364814483
SYD_201811_b9_b14,0.6609238052486465,0.5088591110044896,0.39420698327353143
SYD_201811_b9_b15,0.6688768498493314,0.5036199840937026,0.4012546682120058
SYD_201811_b9_b16,0.6659209912022,0.4861127458056206,0.3724110021913114
SYD_201811_b10_b11,0.6629042489021508,0.49550838958011445,0.41321288586868293
SYD_201811_b10_b12,0.7782894421185365,0.3923560324466303,0.3848736956769493
SYD_201811_b10_b13,0.6748613922784732,0.4945823560529256,0.4176860069171904
SYD_201811_b10_b14,0.6735006164881511,0.49026167064828474,0.3906732691709418
SYD_201811_b10_b15,0.675115810347884,0.4966214565176483,0.3603715021100774
SYD_201811_b10_b16,0.6760102684461354,0.47116178063856573,0.3310701515409501
SYD_201811_b11_b12,0.7072131169308261,0.45912417765187225,0.36788281226892244
SYD_201811_b11_b13,0.682486000534913,0.48237723402289856,0.3605724944972628
SYD_201811_b11_b14,0.6710485880235529,0.4908956114898166,0.37419621600768993
SYD_201811_b11_b15,0.6659342986494958,0.49718527419756214,0.38851997422546913
SYD_201811_b11_b16,0.6567879858341835,0.5061072693072123,0.3910628602626965
SYD_201811_b12_b13,0.719427864570572,0.459488596332605,0.3723944030107747
SYD_201811_b12_b14,0.7449941992136707,0.43966634909547064,0.375478494337817
SYD_201811_b12_b15,0.761650758413908,0.43761718358561474,0.377979492185895
SYD_201811_b12_b16,0.8067910956732071,0.4325217039604628,0.38318204241265946
SYD_201811_b13_b14,0.6813471882701919,0.49294457294322236,0.3768408722445272
SYD_201811_b13_b15,0.6739898008870044,0.4974854888764465,0.38699526120725514
SYD_201811_b13_b16,0.6596300745505669,0.5083096957934748,0.4046338019997292
SYD_201811_b14_b15,0.6764281870622324,0.4851244269882703,0.35976737929179003
SYD_201811_b14_b16,0.6608194578846296,0.4989060186658986,0.3815401049744754
SYD_201811_b15_b16,0.6775006444370257,0.48687651283293437,0.3589300940892035
SYD_201812_b8_b9,1.1412618439119178,0.4821380141530589,0.3266561600574415
SYD_201812_b8_b10,1.1152140213336892,0.5119943811956251,0.39841640810734574
SYD_201812_b8_b11,1.0819521933608944,0.5447721795939315,0.4388714879437047
SYD_201812_b8_b12,1.1108472976601798,0.5454812022759594,0.4482998540170911
SYD_201812_b8_b13,1.0828683386807185,0.5457158886563539,0.4325539089752335
SYD_201812_b8_b14,1.0975852849232368,0.5420555030271968,0.437955190824325
SYD_201812_b8_b15,1.1101080662361293,0.5360458968372035,0.4351583034921101
SYD_201812_b8_b16,1.126981314896627,0.5277441653618136,0.4194830729290943
SYD_201812_b9_b10,1.1246443895693354,0.5168916562304626,0.4109674714451499
SYD_201812_b9_b11,1.0906253844373117,0.548860826446281,0.44791588700369117
SYD_201812_b9_b12,1.1076531561389245,0.5520647201026565,0.44234649536305576
SYD_201812_b9_b13,1.0934192419622446,0.5503167937754827,0.45200818465067977
SYD_201812_b9_b14,1.1070872055174272,0.5450054974488044,0.4428993484511426
SYD_201812_b9_b15,1.1174022184822854,0.5388130017073814,0.4207246501455935
SYD_201812_b9_b16,1.1292935142966884,0.5333878364905283,0.4246204411745058
SYD_201812_b10_b11,1.0920780237124152,0.5548081454211377,0.44623975184733217
SYD_201812_b10_b12,1.109919589779584,0.5524132522806571,0.4254318213734027
SYD_201812_b10_b13,1.0956892630161519,0.5562038893860819,0.4456379710048473
SYD_201812_b10_b14,1.1136114169982578,0.5494743661474131,0.4368696993398754
SYD_201812_b10_b15,1.122631458726959,0.5402488072792441,0.4282488937054096
SYD_201812_b10_b16,1.131290086152384,0.5173000432300392,0.39770690853234836
SYD_201812_b11_b12,1.1055704093278034,0.5262832485665391,0.4221131746777803
SYD_201812_b11_b13,1.1090344493677908,0.5300509845178882,0.4219608565426934
SYD_201812_b11_b14,1.1041575198217675,0.5360650938661026,0.4284165564493398
SYD_201812_b11_b15,1.1016739095769785,0.5416825283189678,0.4310612552031909
SYD_201812_b11_b16,1.0915700521423848,0.5492661851181362,0.43706961325966853
SYD_201812_b12_b13,1.1059285838284298,0.5255552132980815,0.42088708643868616
SYD_201812_b12_b14,1.113214465350687,0.5320109662963372,0.42597721875006356
SYD_201812_b12_b15,1.1131148043289607,0.5361949884608247,0.42810146610091127
SYD_201812_b12_b16,1.114733208009013,0.5480894621317439,0.4314760680745258
SYD_201812_b13_b14,1.1038550949809465,0.531802443033122,0.42686248857506404
SYD_201812_b13_b15,1.1005651729398156,0.5393557844912461,0.42948394575119003
SYD_201812_b13_b16,1.0899637678761755,0.550450326064969,0.43461025853220286
SYD_201812_b14_b15,1.1188080905753048,0.5245965177532189,0.4201010899094087
SYD_201812_b14_b16,1.1067684822437025,0.5378028728704299,0.43121207168927383
SYD_201812_b15_b16,1.1243679742040864,0.5261884853204524,0.4228695236690234
SYD_201901_b8_b9,0.7080180831350464,0.4159094485764528,0.36275917065390745
SYD_201901_b8_b10,0.690032069360486,0.43942449188683,0.3774996624313941
SYD_201901_b8_b11,0.6670107126192452,0.47629754691369014,0.40353103727737555
SYD_201901_b8_b12,0.688614027123939,0.47848110119596177,0.38433315898583204
SYD_201901_b8_b13,0.6687795846548501,0.4778735892436932,0.4049584993986593
SYD_201901_b8_b14,0.6811053049390979,0.4735457528761846,0.3912784827398008
SYD_201901_b8_b15,0.6879666640692184,0.46525756153873343,0.38708807122509375
SYD_201901_b8_b16,0.6986973753281481,0.45684635065877044,0.37705369716513254
SYD_201901_b9_b10,0.7054661788955044,0.43785950363744375,0.3671620198879213
SYD_201901_b9_b11,0.6781451832816676,0.48941683959412435,0.3949260171995951
SYD_201901_b9_b12,0.690462549886012,0.4881018318246688,0.3891098258693552
SYD_201901_b9_b13,0.6808746739615491,0.4921823936294949,0.3945303551694538
SYD_201901_b9_b14,0.6935074925473769,0.4846290618948328,0.3858048856874413
SYD_201901_b9_b15,0.7027408682071398,0.47269822483329554,0.375471111629172
SYD_201901_b9_b16,0.7159157879351755,0.4569649312360953,0.3597150999889462
SYD_201901_b10_b11,0.6837816893702015,0.49588446547976844,0.38593882070979957
SYD_201901_b10_b12,0.6922392788741465,0.49385341955514783,0.38084583990504217
SYD_201901_b10_b13,0.6880305840388404,0.4976779604844279,0.39274574288065306
SYD_201901_b10_b14,0.7031609884265634,0.4837039217966385,0.36954265478233206
SYD_201901_b10_b15,0.7119504031020804,0.4699665738645015,0.3472348166120683
SYD_201901_b10_b16,0.7178417206591267,0.44539711814500726,0.3327152640196372
SYD_201901_b11_b12,0.7227092827233778,0.45631878791511715,0.35254364848216174
SYD_201901_b11_b13,0.7046585095656895,0.45780014466374125,0.35666980725228054
SYD_201901_b11_b14,0.6956739461271559,0.4699676893761563,0.37234683824642373
SYD_201901_b11_b15,0.6910899715650475,0.48005097547159,0.37645480516634555
SYD_201901_b11_b16,0.6819387482054375,0.48730855841845294,0.38663971385273427
SYD_201901_b12_b13,0.7196577680358107,0.45497189609763705,0.3527782053255349
SYD_201901_b12_b14,0.6970921858662916,0.4624235557272114,0.36823524498796983
SYD_201901_b12_b15,0.6945172219073662,0.46992787899911515,0.37334604447804864
SYD_201901_b12_b16,0.6947273584638287,0.48532764080290697,0.37438775996784673
SYD_201901_b13_b14,0.6961228302987006,0.46691846182128116,0.36551492294277704
SYD_201901_b13_b15,0.6908612140316377,0.480366540198274,0.3695447867605756
SYD_201901_b13_b16,0.6819173146510447,0.48693057509474774,0.3822670009101215
SYD_201901_b14_b15,0.7086246776235194,0.45988074217732366,0.3509085797531116
SYD_201901_b14_b16,0.697727395974974,0.47246631378974385,0.3677206683947082
SYD_201901_b15_b16,0.7104694596391768,0.4588811524988277,0.35960393663640655

SYD_201902_b8_b9,0.3833337467026258,0.3521702596872062,0.3364917329568593
SYD_201902_b8_b10,0.37372258251941515,0.36197517148312364,0.36498820770429985
SYD_201902_b8_b11,0.36606409307395504,0.4178581790602619,0.39346615050514583
SYD_201902_b8_b12,0.3851366083457628,0.4288836870417164,0.33776817462612924
SYD_201902_b8_b13,0.364329994448452,0.4158885770422595,0.40039856968978416
SYD_201902_b8_b14,0.3719856197774258,0.4102723054937454,0.39098666143585714
SYD_201902_b8_b15,0.3742784598324925,0.3999467873664339,0.38840547305396655
SYD_201902_b8_b16,0.37962704210717163,0.389173360917658,0.3774351363927911
SYD_201902_b9_b10,0.387172515680282,0.37658505815099547,0.29802112658716157
SYD_201902_b9_b11,0.3742139880096599,0.4338023503658045,0.38518684534913616
SYD_201902_b9_b12,0.38959037888690695,0.4424303471309029,0.23775025945368625
SYD_201902_b9_b13,0.37347971614026515,0.43739043919078985,0.37847599410490773
SYD_201902_b9_b14,0.3805352060720217,0.42723739861929755,0.37575558259611275
SYD_201902_b9_b15,0.38358245363842297,0.4136998663157665,0.3734459140454474
SYD_201902_b9_b16,0.39020417277288827,0.3999810230847631,0.30028391033518365
SYD_201902_b10_b11,0.3786465117207602,0.4348839924907256,0.32911922239188907
SYD_201902_b10_b12,0.3917847832338577,0.4430608598453735,0.23501686892592777
SYD_201902_b10_b13,0.38001493505776696,0.43837209723646403,0.3474734728110741
SYD_201902_b10_b14,0.38788946755213427,0.43482296493570477,0.3311966622416082
SYD_201902_b10_b15,0.3905402518203542,0.4157883046603667,0.27304247830783707
SYD_201902_b10_b16,0.39309979683963847,0.3933510298430446,0.21456176807597444
SYD_201902_b11_b12,0.3906400428582975,0.400197578312397,0.24222986763076568
SYD_201902_b11_b13,0.3885214190970488,0.39596878453125145,0.2891499723853994
SYD_201902_b11_b14,0.3852498555409234,0.40247972572730595,0.3110481404991755
SYD_201902_b11_b15,0.382763694954309,0.4141322140739976,0.3263251774012176
SYD_201902_b11_b16,0.37774779943687853,0.4262487695452169,0.3294076233767228
SYD_201902_b12_b13,0.39122332626908835,0.4003042297138979,0.2267043568360377
SYD_201902_b12_b14,0.3926532367272416,0.4079972915486013,0.23327520055807463
SYD_201902_b12_b15,0.39238627940284093,0.41833523176021875,0.2400118553645525
SYD_201902_b12_b16,0.3911739797869346,0.43192890501629544,0.2347778277602839
SYD_201902_b13_b14,0.38574443750371284,0.40189885451190893,0.3039679105222862
SYD_201902_b13_b15,0.38241794798780704,0.41404813506631366,0.31979282278949317
SYD_201902_b13_b16,0.37541259702359314,0.42390425190701303,0.3300293860694146
SYD_201902_b14_b15,0.39037624962655787,0.4026636102914101,0.22590355596123593
SYD_201902_b14_b16,0.38377014733270387,0.41537291144233746,0.3161399984547632
SYD_201902_b15_b16,0.39064345297371594,0.3954634201020882,0.29656253028785573
NT_201811_b8_b9,0.42783403306167583,0.4613086721967458,0.45392864206661104
NT_201811_b8_b10,0.42137986619074147,0.4771115229331661,0.472075887330197
NT_201811_b8_b11,0.41511912899989833,0.5014621735112567,0.4951283377896437
NT_201811_b8_b12,0.415677638314004,0.4979461738709208,0.4986465607313009
NT_201811_b8_b13,0.4185854306657219,0.5131171187808935,0.49712175475358633
NT_201811_b8_b14,0.4200447195498904,0.5049616878837759,0.47867210052640513
NT_201811_b8_b15,0.42189184114396616,0.4963632410100296,0.4660150475476028
NT_201811_b8_b16,0.42617869478287956,0.4804784137974096,0.4514844030432232
NT_201811_b9_b10,0.4236207442543057,0.4727412109894517,0.46596325898153657
NT_201811_b9_b11,0.41489099982762023,0.5028282517538638,0.49039022520044667
NT_201811_b9_b12,0.415529835694189,0.4975214438720831,0.4930878257616692
NT_201811_b9_b13,0.4191371958762001,0.5102832031790483,0.48323193032882106
NT_201811_b9_b14,0.4204072946313941,0.5092742942792905,0.4787314713221183
NT_201811_b9_b15,0.42260547199270077,0.5000105561693773,0.4560321704009408
NT_201811_b9_b16,0.4266509866181345,0.472457981981883,0.43446824338422735
NT_201811_b10_b11,0.41554565963894496,0.48060524568958174,0.4802038081745172
NT_201811_b10_b12,0.4160682923922941,0.4976460923553294,0.49158184064756366
NT_201811_b10_b13,0.4203380152782559,0.49030873352940285,0.47912517086506545
NT_201811_b10_b14,0.42240532284200655,0.4958247760660232,0.4553704420049176
NT_201811_b10_b15,0.4236057856760196,0.4871877720293915,0.4434203325901236
NT_201811_b10_b16,0.4249108752676144,0.46878495598046394,0.4520540730337078
NT_201811_b11_b12,0.4198671988700182,0.48329486520107734,0.4709769467597808
NT_201811_b11_b13,0.42077816714905925,0.48547399329400215,0.47372427607265427
NT_201811_b11_b14,0.41717170419594474,0.49297762637565773,0.47785187697022224
NT_201811_b11_b15,0.4164265805639911,0.4983237595358063,0.4815955260026132
NT_201811_b11_b16,0.4160174774638636,0.49579823116547933,0.4879634971416583
NT_201811_b12_b13,0.41695617759555026,0.48332452843096635,0.47628302302380116
NT_201811_b12_b14,0.41694779209183797,0.48742486039339095,0.4794365617931319
NT_201811_b12_b15,0.4163873188606109,0.49080035379299,0.4859317859541757
NT_201811_b12_b16,0.415294482200211,0.49019577608848736,0.4887968243226514
NT_201811_b13_b14,0.42032951665420804,0.4884389141452365,0.4763073985029072
NT_201811_b13_b15,0.419292545427761,0.4995836580088026,0.48073075176971064
NT_201811_b13_b16,0.4175122036789466,0.4710153424734638,0.48596116786506144
NT_201811_b14_b15,0.42232648670413203,0.4791976628151937,0.4549354297355266
NT_201811_b14_b16,0.4203945845692824,0.4825933712589662,0.48036143040117296
NT_201811_b15_b16,0.42475705727345614,0.49106495831422364,0.44981421134659066
NT_201812_b8_b9,2.31853919449357,0.5484575760567182,0.5841426869336029
NT_201812_b8_b10,2.3016861417593617,0.5450553923659269,0.5935760517332425
NT_201812_b8_b11,2.2705774461809187,0.5974209750069757,0.6067229497789562
NT_201812_b8_b12,2.270247034932647,0.5956748661611638,0.6022927226772371
NT_201812_b8_b13,2.2648982366155948,0.5999248172118463,0.603802963038094
NT_201812_b8_b14,2.2935213291977763,0.6026264967662625,0.5983857133431569
NT_201812_b8_b15,2.306598955392799,0.5901618990091376,0.594204304203981
NT_201812_b8_b16,2.321433887469535,0.5720993958342289,0.584947379669181
NT_201812_b9_b10,2.3107727960507694,0.5408137719453966,0.5878399786064548
NT_201812_b9_b11,2.272762090195081,0.5992437795017599,0.6064082453193367
NT_201812_b9_b12,2.276145125963136,0.5944545761357901,0.5997566826546553
NT_201812_b9_b13,2.2653524700569148,0.5999548541223829,0.601502341166811
NT_201812_b9_b14,2.300029536579004,0.6006616218056926,0.5926891758336198
NT_201812_b9_b15,2.3087304001376125,0.5837038953084384,0.5875398619737007
NT_201812_b9_b16,2.322783136221979,0.5387461567097299,0.573560360204564
NT_201812_b10_b11,2.2725068477131787,0.6152280699006212,0.6041892148181104
NT_201812_b10_b12,2.276876145395734,0.6014218967041294,0.5982115180880243
NT_201812_b10_b13,2.2696683981561394,0.6110789684328799,0.6027139332554213
NT_201812_b10_b14,2.303121091433238,0.5883475970632027,0.5911224298226281
NT_201812_b10_b15,2.307978538455215,0.603543891823477,0.5802562191424617
NT_201812_b10_b16,2.3158556985454832,0.5307350763615076,0.5793824010195059
NT_201812_b11_b12,2.285756925278168,0.5728428623350069,0.5899016610197504
NT_201812_b11_b13,2.2902714442374923,0.571342538518631,0.5926510999840884
NT_201812_b11_b14,2.2838520159800697,0.5774626665575167,0.5979714776228354
NT_201812_b11_b15,2.2754281405932137,0.594607588631303,0.5996068411400728
NT_201812_b11_b16,2.2788367761844532,0.5923443776026202,0.6005842967675857
NT_201812_b12_b13,2.2759425625408802,0.5720183286160055,0.5904739842308615
NT_201812_b12_b14,2.277922957057839,0.5704924932121803,0.5947907537632071
NT_201812_b12_b15,2.269711627086982,0.5809941212010779,0.596228303045549
NT_201812_b12_b16,2.267884322129757,0.5835683299466912,0.5979598548439684
NT_201812_b13_b14,2.263446093570872,0.5673826183065959,0.5963139869191207
NT_201812_b13_b15,2.2538740104927646,0.5925877077288315,0.6003933733515817
NT_201812_b13_b16,2.255981289755796,0.5801840422164091,0.5997527310936367
NT_201812_b14_b15,2.2937339380841846,0.5606189943624038,0.5913415437349064
NT_201812_b14_b16,2.290542555718218,0.5994102138022015,0.5973918584070798
NT_201812_b15_b16,2.3054494127716048,0.5668573764266943,0.590977193274513
NT_201901_b8_b9,1.7395317095677587,0.5470334843327098,0.5455027142285765

NT_201901_b8_b10,1.727159725331522,0.5526327866732924,0.5588206009328781
NT_201901_b8_b11,1.7056875406167737,0.5861651433878196,0.5728665023384533
NT_201901_b8_b12,1.6454990134220562,0.5862504175155085,0.574554850905543
NT_201901_b8_b13,1.7103900701009642,0.5911425633407082,0.5709766802570557
NT_201901_b8_b14,1.7242180460417702,0.5821752105275148,0.5663141710817589
NT_201901_b8_b15,1.7305882820415897,0.5799121596416383,0.5563122661163716
NT_201901_b8_b16,1.7375171073625693,0.5759318864238829,0.5503644306000025
NT_201901_b9_b10,1.7342029545658306,0.5503484464434323,0.5563233556117043
NT_201901_b9_b11,1.7066182000202479,0.5863011811631745,0.5708007705209807
NT_201901_b9_b12,1.6487014384855294,0.5874953626532881,0.5719655559138306
NT_201901_b9_b13,1.7112274693623193,0.5916920577619196,0.5692678062135158
NT_201901_b9_b14,1.7268967471358148,0.58878941587423,0.561699648663519
NT_201901_b9_b15,1.730552679665194,0.5778473670595463,0.5512341097890882
NT_201901_b9_b16,1.7390846685217816,0.5776871982078038,0.5384263391454108
NT_201901_b10_b11,1.7077891661376392,0.5895091083501148,0.5705201772264294
NT_201901_b10_b12,1.6481653859592158,0.5903401480726723,0.5711396928559136
NT_201901_b10_b13,1.7143549972228513,0.5891890343752307,0.5666443022550234
NT_201901_b10_b14,1.7291633060104425,0.5757685191550089,0.5547766868715087
NT_201901_b10_b15,1.7277510617884424,0.5704744091140962,0.5429372139982314
NT_201901_b10_b16,1.735674940857097,0.563328988580433,0.543579535883697
NT_201901_b11_b12,1.6597859243434299,0.5789364679512795,0.5601240643228915
NT_201901_b11_b13,1.7238455561603818,0.5708440604953514,0.5574984658493697
NT_201901_b11_b14,1.7125016575451983,0.5778916865449225,0.561411947071922
NT_201901_b11_b15,1.7064350646666642,0.5823597227143649,0.5637657375384516
NT_201901_b11_b16,1.7092982183678926,0.5832693307111451,0.5659998127399269
NT_201901_b12_b13,1.6586039643454467,0.5762850865515964,0.562530436374828
NT_201901_b12_b14,1.6550913588050067,0.5807283880897807,0.5654958509185067
NT_201901_b12_b15,1.6510324869045543,0.5851500628975661,0.5671098606194521
NT_201901_b12_b16,1.6501107853426307,0.5869216964714309,0.5679458607035716
NT_201901_b13_b14,1.7125108715328892,0.5728120321747427,0.5612557398696664
NT_201901_b13_b15,1.7047061712948028,0.5910258853324646,0.5635808799247155
NT_201901_b13_b16,1.7065188884512708,0.5792057353486625,0.5658823412461728
NT_201901_b14_b15,1.7281978030431886,0.5656313612187276,0.5549341077529027
NT_201901_b14_b16,1.72525498797263,0.5742269579198725,0.5588418122141355
NT_201901_b15_b16,1.7377681601090034,0.5578339398016967,0.5512898438410146
NT_201902_b8_b9,1.0804263034230561,0.4902908683275071,0.4783528789348382
